# Static LCA

This is the forth notebook of a series of 5 (see below the names of full list) intended to teach you the functionalities of the pyhton-based LCA software Brightway 2.5 (BW25) that you will use to carry out your group assignment. For a more nuanced undertsanding of the principles and working mechanisms of BW25 you are referred to [its documentation](https://docs.brightway.dev/en/latest/index.html).

**Full list of notebooks of the series:**
- 1.Getting started
- 2a.Background
- 2b.Foreground
- 3.StaticLCA
- 4.StochasticLCA

## Pre-requesites:
- Having read the tutorial _Computational Settings and Guidelines_ that you can find on Canvas -> [Module 'General documents & Information'](https://canvas.utwente.nl/courses/15132/modules#:~:text=General%20documents%20%26%20information-,General,-documents%20%26%20information)
- Having installed Brightway25 and all relevant packages (make sure to do so in the correct virtual environment) and eventually having had a first contact with its documentation.
- Knowledge of the foundations of life cycle assessment (LCA). If you did not attended the Lecture 3- Pavement Life cycle assessment (LCA) or need a refresher, please have a look at its slides that you can find on Canvas.
- Access to the ecoinvent database version 3.9.1.
- Acess to the foreground database defined in the excel file 'Foreground.xlsx'
- Having gone through the previous notebooks.
- Familiarity with the Python programming language and the Jupyter Lab environment. However, this is **not** a hard requirement since most methods are quite intuitive and will be explained throughput the tutorial. Nevertheless, In case you are a total beginner, or need a refresher, please read the tutorial _Computational Settings and Guidelines_ that you can find on Canvas -> [Module 'General documents & Information'](https://canvas.utwente.nl/courses/15132/modules#:~:text=General%20documents%20%26%20information-,General,-documents%20%26%20information)
  

## Learning Objectives
- Learn how to perform static (i.e. deterministic, or in other words, ignoring uncertainties) LCAs:
    * Single product and single impact categories
    * Single product and multiple impact categories
    * Comparative analysis with multiple impact categories
- Learn how to perform contribution analysis (activities and environmental flows)


## Content

[3.1. Single-LCA: LCA with an activity from the ecoinvent database <br>](#single_lca1)
[3.2. Single-LCA: LCA with an activity from the foreground database related to a pavement system <br>](#single_lca2)
[3.3. Single-LCA: Revisiting my second LCA with multiple impact categories <br>](#single_lca3)
[3.4. Single-LCA: LCA of a pavement system encompassing the pavement life cycle phases A1-A5 <br>](#single_lca4)
[3.5. Comparative-LCA of 2 pavement systems with MultiLCA](#comparative_lca)



In [16]:
# Import Python packages
from IPython.display import display
import pandas as pd
import numpy as np

# Import BW25 packages.
import bw2analyzer as bwa
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import bw2analyzer as bwa
from bw2data.query import Filter, Query


In [17]:
#Import from the python file 'project_details.py' the variables with the project name and background db. For your group assigment you can update that file with the name of your project and background databases
from project_details import ei_name, project_name
print(ei_name)
print(project_name)

ecoinvent-391-cutoff
2024_STI_BW25_Lecture


In [19]:
bd.projects.set_current(project_name)
bd.databases

Databases dictionary with 3 object(s):
	asphalt
	biosphere3
	ecoinvent-391-cutoff

In [20]:
# Is the background database name the same as the one we wrote in `project_details.py`?
assert ei_name in bd.databases

Let's define and assign the name of the foreground database to a variable called "fg_name".
REMINDER: The name must be equal to the content of the cell B1 in the excel file that contains our foreground database: in this example the file "Foreground.xlsx". You can select the name that you want (for instance: group assignment). The only (and mandatory) requirement is that it is equal to the content of the cell B1 in the excel file. 

In [21]:
fg_name = "asphalt"

In [22]:
# Assign the foreground database to a variable:
fgdb = bd.Database(fg_name)

## 3.1) Single-LCA: LCA with an activity from the ecoinvent database <a name="single_lca1"></a>

To perform a LCA, we construct a technosphere matrix, which describes the inputs needed to produce different products (e.g. cars need metal and electricity), and a biosphere matrix, which describes the emissions and resource consumption associated with the production of each product (e.g. car manufacturing releases air emissions). These two matrices come from the life cycle inventory database(s). We also have a functional unit, which is what we are trying to assess, e.g. one car. We then calculate the life cycle inventory (LCI) by first solving the linear system of the technosphere matrix and the functional unit, and then by multiplying the biosphere matrix. To do the life cycle impact assessment (LCIA), we multiply the LCI by a matrix of characterization factors (CFs). A CF is a biophysical quantity that indicates the impact on a certain environmental mechanism per unit of emissions or resource extraction. Adapted from [here](https://github.com/brightway-lca/brightway25/blob/main/notebooks/Getting%20Started%20with%20Brightway2.ipynb).

The full details of the matrix-based LCA is out of the scope of this course. However, if you want learn more about it you can watch [this](https://www.youtube.com/watch?v=3GDfNksiY0s) video and/or read [this](https://link.springer.com/chapter/10.1007/978-3-031-49317-1_6) book chapter.

Brightway has a so-called `LCA` object. It is instantiated using `LCA(args)`.
The only required argument is a functional unit, described by a dictionary with keys = activities and values = amounts.
A second argument that is often passed is an LCIA method, passed using the method tuple.

To illustrate the description above, let's select a simple product system corresponding to an activity that we looked into before, namely the activity 'limestone production, crushed, washed' (kilogram, RoW, None), existing in the ecoinvent database. Its code is '570143a9c955c81d5f8bc1e9bd8d22a1'

In [23]:
#Check existing databases
bd.databases

Databases dictionary with 3 object(s):
	asphalt
	biosphere3
	ecoinvent-391-cutoff

In [24]:
# Is the background database name the same as the one we wrote in `project_details.py`?
assert ei_name in bd.databases

#Assign ei db to a variable
eidb = bd.Database(ei_name)

In [25]:
#Select the activity we are interested in
my_act = eidb.get('570143a9c955c81d5f8bc1e9bd8d22a1')
print(my_act)
my_act.get('reference product')

'limestone production, crushed, washed' (kilogram, RoW, None)


'limestone, crushed, washed'

In [26]:
my_act.as_dict()

{'comment': "Infrastructure data are estimated based on a site visit and sketches of the process. The value of the infrastructure is normalized with a annual production capacity of about 195'000 tons of product per year. The estimated lifespan of the machines is 25 years. Assumed size partition of dust emissions for crushing, screening and transport processes: 50% >10µm (PM10), 45% between 10 µm and 2.5 µm, 5%<2.5 µm. Limestone, crushed, washed is a pre-product for the milling process. 50% of the electricity consumption is covered by the grid and 50% by a hydropower plant nearby. The infrastructure for electricity transportation from power plant to KFN is not included. 50% of the electricity consumption is covered by the grid and 50% by a hydropower plant nearby. The infrastructure for electricity transportation from power plant to KFN is not included.\nGeography:  data are from only one company in Switzerland (KFN), for some exchanges RER-modules have been used as proxy\nTechnology:  

### Functional unit

The first step consists of defining the functional unit (FU). This is done by using a dictionary with keys = activities and values = amounts. In this examples, let's assume we are interested in the production of 1 unit of the reference product. The definition of the FU and respective amount is done as follows:

In [27]:
functional_unit = {my_act: 1}
functional_unit

{'limestone production, crushed, washed' (kilogram, RoW, None): 1}

The next step consists of selecting the LCIA method to be used for calculating the environmental impact scores. In the notebook '2a_Background' we explored the different LCIA methods that come with the database 'biosphere3'. In this example, we are going to use the method 'CML v4.8 2016', and particularly the impact category 'climate change'.

In [28]:
CML_methods = [m for m in bd.methods if m[0] == "CML v4.8 2016"]
CML_methods

[('CML v4.8 2016',
  'acidification',
  'acidification (incl. fate, average Europe total, A&B)'),
 ('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)'),
 ('CML v4.8 2016',
  'ecotoxicity: freshwater',
  'freshwater aquatic ecotoxicity (FAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: marine',
  'marine aquatic ecotoxicity (MAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: terrestrial',
  'terrestrial ecotoxicity (TETP inf)'),
 ('CML v4.8 2016',
  'energy resources: non-renewable',
  'abiotic depletion potential (ADP): fossil fuels'),
 ('CML v4.8 2016', 'eutrophication', 'eutrophication (fate not incl.)'),
 ('CML v4.8 2016', 'human toxicity', 'human toxicity (HTP inf)'),
 ('CML v4.8 2016',
  'material resources: metals/minerals',
  'abiotic depletion potential (ADP): elements (ultimate reserves)'),
 ('CML v4.8 2016',
  'ozone depletion',
  'ozone layer depletion (ODP steady state)'),
 ('CML v4.8 2016',
  'photochemical oxidant formation',
  'photochemical oxidation (h

The impact category we are interested in is defined as following:

In [29]:
CC_method = ('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)')

### LCIA scores (i.e. results) calculation

Next, we instantiate our LCA object. Let's call it 'myFirstLCA':

In [30]:
myFirstLCA = bc.LCA(functional_unit, CC_method)

And do the LCI and LCIA calculations by running the cells below:

In [31]:
myFirstLCA.lci()
myFirstLCA.lcia()

Finally, we can print the LCA score:

In [32]:
myFirstLCA.score

0.0031468186429639317

Or in a more complete way:

In [33]:
print("The score is {:f} {} for impact category {}".format(myFirstLCA.score, 
                                                 bd.methods.get(CC_method).get('unit'),
                                                 CC_method[1]))

The score is 0.003147 kg CO2-Eq for impact category climate change


To see what else you can do with the lca-object, press tab in the next cell:

In [77]:
myFirstLCA.

SyntaxError: invalid syntax (619557069.py, line 1)

We can also get dataframes the for LCA calculation results. 

In [34]:
df = myFirstLCA.to_dataframe()
df

,row_index,col_index,amount,row_id,col_id,row_database,row_code,row_name,row_location,row_unit,row_type,row_categories,row_product,col_database,col_code,col_name,col_location,col_unit,col_type,col_reference_product
0,689,14277,1.576131e-03,1171,18987,biosphere3,349b29d1-3e58-4c66-98b9-9d1a076efd2e,"Carbon dioxide, fossil",None,kilogram,emission,air,None,ecoinvent-391-cutoff,0c8f3330261952e02d86355874fd2dd8,"diesel, burned in building machine",GLO,megajoule,process,"diesel, burned in building machine"
1,829,1853,1.647161e-04,1649,6563,biosphere3,70ef743b-3ed5-4a6d-b192-fb6d62378555,"Methane, fossil",None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,5bd4d1d3ea2ad8b6f9288db105968f92,natural gas venting from petroleum/natural gas...,GLO,cubic meter,process,"natural gas, vented"
2,82,3031,1.600458e-04,112,7741,biosphere3,f9749677-9c9f-4678-ab55-c607dfdc2cb9,"Carbon dioxide, fossil",None,kilogram,emission,air::urban air close to ground,None,ecoinvent-391-cutoff,61aa8615096f6a0ba6786e06afe605a1,"heat production, anthracite, at stove 5-15kW",RoW,megajoule,process,"heat, central or small-scale, other than natur..."
3,689,3883,7.391740e-05,1171,8593,biosphere3,349b29d1-3e58-4c66-98b9-9d1a076efd2e,"Carbon dioxide, fossil",None,kilogram,emission,air,None,ecoinvent-391-cutoff,b1b3b47eb0e3d7b0317ae9e38b32c247,"diesel production, petroleum refinery operation",RoW,kilogram,process,diesel
4,686,11949,5.952757e-05,1168,16659,biosphere3,aa7cac3a-3625-41d4-bc54-33e2cf11ec46,"Carbon dioxide, fossil",None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,c7bceed2927df265f4d635437756b3d9,"sweet gas, burned in gas turbine",GLO,megajoule,process,"sweet gas, burned in gas turbine"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,686,4068,9.297220e-07,1168,8778,biosphere3,aa7cac3a-3625-41d4-bc54-33e2cf11ec46,"Carbon dioxide, fossil",None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,a5a6266daa144325e454cafbc34bac40,"electricity production, natural gas, conventio...",KR,kilowatt hour,process,"electricity, high voltage"
196,686,12510,9.242229e-07,1168,17220,biosphere3,aa7cac3a-3625-41d4-bc54-33e2cf11ec46,"Carbon dioxide, fossil",None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,9bcc79119f9fe4087c7c164a7170b093,"electricity production, hard coal",IN-HR,kilowatt hour,process,"electricity, high voltage"
197,82,3767,9.171451e-07,112,8477,biosphere3,f9749677-9c9f-4678-ab55-c607dfdc2cb9,"Carbon dioxide, fossil",None,kilogram,emission,air::urban air close to ground,None,ecoinvent-391-cutoff,4aab528dfde9fd07ce6f1aee4304cf95,"electricity production, oil",MX,kilowatt hour,process,"electricity, high voltage"
198,686,20037,9.157838e-07,1168,24747,biosphere3,aa7cac3a-3625-41d4-bc54-33e2cf11ec46,"Carbon dioxide, fossil",None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,aa4f5d6d66ff569b28464f715e5b251b,"synthetic fuel production, from coal, high tem...",ZA,kilogram,process,"diesel, low-sulfur"


By default, this method looks at the `characterized_inventory` matrix, and sorts by the top 200 values (using absolute value).

The columns labels are a bit different, as we don't have target and source but instead matrix rows and columns. The meaning of these rows and columns changes from matrix to matrix. The same pattern with 'row_product', 'col_reference_product', and 'row_categories' applies though.

In [79]:
df.columns

Index(['row_index', 'col_index', 'amount', 'row_id', 'col_id', 'row_database',
       'row_code', 'row_name', 'row_location', 'row_unit', 'row_type',
       'row_categories', 'row_product', 'col_database', 'col_code', 'col_name',
       'col_location', 'col_unit', 'col_type', 'col_reference_product'],
      dtype='object')

We can get dataframes for any matrix. In standard LCA, the matrices are:

- inventory
- technosphere_matrix
- biosphere_matrix
- characterization_matrix
- characterized_inventory

Note that for other matrices you will need to specify the row and column mapping dictionaries, see the docstring.

For instance, let's assume we want to get the **'biosphere_matrix'**

In [80]:
myFirstLCA.to_dataframe(matrix_label='biosphere_matrix')

,row_index,col_index,amount,row_id,col_id,row_database,row_code,row_name,row_location,row_unit,row_type,row_categories,row_product,col_database,col_code,col_name,col_location,col_unit,col_type,col_reference_product
0,333,15850,3.520400e+10,526,20560,biosphere3,379ba5c9-5c3a-43d0-8e2d-605ad9c39e46,"Occupation, mineral extraction site",None,square meter-year,natural resource,natural resource::land,None,ecoinvent-391-cutoff,121b570a5c9b8421b33416cb10cfbb27,"mine construction, gold",US,unit,process,"mine infrastructure, gold"
1,332,12243,3.500000e+09,524,16953,biosphere3,69dfa439-8e4e-4cae-bb0c-85a8aa8b9a73,"Occupation, lake, artificial",None,square meter-year,natural resource,natural resource::land,None,ecoinvent-391-cutoff,6d8f423f2f91b29a4f1b82cf498a9858,port facilities construction,RoW,unit,process,port facilities
2,332,12091,3.500000e+09,524,16801,biosphere3,69dfa439-8e4e-4cae-bb0c-85a8aa8b9a73,"Occupation, lake, artificial",None,square meter-year,natural resource,natural resource::land,None,ecoinvent-391-cutoff,2fd36ed40e1ad80fc2d07b822f6f283d,port facilities construction,RER,unit,process,port facilities
3,333,13685,2.529500e+09,526,18395,biosphere3,379ba5c9-5c3a-43d0-8e2d-605ad9c39e46,"Occupation, mineral extraction site",None,square meter-year,natural resource,natural resource::land,None,ecoinvent-391-cutoff,bf79c2917ce1c7c6df3f6e244211b257,"mine construction, gold",CA,unit,process,"mine infrastructure, gold"
4,333,14872,2.360000e+09,526,19582,biosphere3,379ba5c9-5c3a-43d0-8e2d-605ad9c39e46,"Occupation, mineral extraction site",None,square meter-year,natural resource,natural resource::land,None,ecoinvent-391-cutoff,f3d7eac4f256e2a49367b2bea0b38b41,"mine infrastructure construction, treatment of...",ZA,unit,process,"mine infrastructure, treatment of sulfidic tai..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,330,7253,7.600000e+06,522,11963,biosphere3,fe9c3a98-a6d2-452d-a9a4-a13e64f1b95b,"Occupation, industrial area",None,square meter-year,natural resource,natural resource::land,None,ecoinvent-391-cutoff,ac028074fbd9bee7e47e0f7646e3dd5d,"nuclear power plant construction, boiling wate...",CH,unit,process,"nuclear power plant, boiling water reactor 1000MW"
196,330,7121,7.600000e+06,522,11831,biosphere3,fe9c3a98-a6d2-452d-a9a4-a13e64f1b95b,"Occupation, industrial area",None,square meter-year,natural resource,natural resource::land,None,ecoinvent-391-cutoff,92028c4cebf1499c7921bbc531b859df,"nuclear power plant construction, boiling wate...",US,unit,process,"nuclear power plant, boiling water reactor 1000MW"
197,330,18759,7.600000e+06,522,23469,biosphere3,fe9c3a98-a6d2-452d-a9a4-a13e64f1b95b,"Occupation, industrial area",None,square meter-year,natural resource,natural resource::land,None,ecoinvent-391-cutoff,bf740244ccab787ac1674ea317bab299,"nuclear power plant construction, boiling wate...",DE,unit,process,"nuclear power plant, boiling water reactor 1000MW"
198,330,18637,7.600000e+06,522,23347,biosphere3,fe9c3a98-a6d2-452d-a9a4-a13e64f1b95b,"Occupation, industrial area",None,square meter-year,natural resource,natural resource::land,None,ecoinvent-391-cutoff,2d2b9c0d438ebf72a85c31cfa574f659,"nuclear power plant construction, boiling wate...",RoW,unit,process,"nuclear power plant, boiling water reactor 1000MW"


In the notebook '2a_background' we saw how the function `print_recursive_calculation` is used to print the supply chain of an activity. This function can be used to transverse the characterized inventory (i.e., LCA scores). NOTE: running this function is usually time consuming.

<span style='background:orange'> Continue based on the website below </span>:
https://github.com/Depart-de-Sentier/teaching-material/blob/main/beginners/Interpretation/Exploration%20functions.ipynb

In [81]:
bwa.print_recursive_calculation(my_act, CC_method)

Fraction of score | Absolute score | Amount | Activity
0001 | 0.003147 |     1 | 'limestone production, crushed, washed' (kilogram, RoW, None)


KeyboardInterrupt: 

### Contribution analysis

Quite often, we are interested in knowing the activities that contribute the most to the environmental impacts score. This task is called _Contribution analysis_. We can do this by creating an instance of the class `ContributionAnalysis()`, and then call the method `annotated_top_processes`. This method has two arguments, the LCA object defined before and the limit number of the most damaging activities we are intersted in, and returns a list of tuples: `(lca score, supply amount, activity)`. The activities are sorted by the absolute value of the LCA socres.

 Let's apply this to our example:

In [82]:
ca = bwa.ContributionAnalysis()
results = ca.annotated_top_processes(myFirstLCA, limit=20) #We are limiting the outcomes to the 20 most damaging activities. You can chose the value you want

print("Impact category ** {} **".format(CC_method[1].title()))
print(results)

Impact category ** Climate Change **
[(0.0015944183825432857, 0.021590835187152404, 'diesel, burned in building machine' (megajoule, GLO, None)), (0.00017092394554533093, 0.0013990020186153372, 'heat production, anthracite, at stove 5-15kW' (megajoule, RoW, None)), (0.00016485683692315156, 1.0055925519126212e-05, 'natural gas venting from petroleum/natural gas production' (cubic meter, GLO, None)), (7.48150504389357e-05, 0.0003821205838620865, 'diesel production, petroleum refinery operation' (kilogram, RoW, None)), (5.9586519317681226e-05, 0.0008897992546849275, 'sweet gas, burned in gas turbine' (megajoule, GLO, None)), (5.269690383317356e-05, 0.00013828361947590734, 'hard coal mine operation and hard coal preparation' (kilogram, CN, None)), (3.4370656413150856e-05, 4.0479878638776275e-05, 'pig iron production' (kilogram, RoW, None)), (3.24820003283498e-05, 0.0004769750559768772, 'treatment of waste natural gas, sweet, burned in production flare' (megajoule, GLO, None)), (2.969778903

We can see the results in a table:

In [83]:
df = pd.DataFrame(results, columns=['CC score', 'Supply amount', 'Activity Name'])
df

,CC score,Supply amount,Activity Name
0,0.001594,0.021591,"[comment, classifications, activity type, acti..."
1,0.000171,0.001399,"[comment, classifications, activity type, acti..."
2,0.000165,0.000010,"[comment, classifications, activity type, acti..."
3,0.000075,0.000382,"[comment, classifications, activity type, acti..."
4,0.000060,0.000890,"[comment, classifications, activity type, acti..."
5,0.000053,0.000138,"[comment, classifications, activity type, acti..."
6,0.000034,0.000040,"[comment, classifications, activity type, acti..."
7,0.000032,0.000477,"[comment, classifications, activity type, acti..."
8,0.000030,0.000258,"[comment, classifications, activity type, acti..."
9,0.000024,0.000018,"[comment, classifications, activity type, acti..."


Similarly, we can also get a list with the most damaging biosphere flows in the LCA. To do so, we can use the method `annotated_top_emissions`. This method has three arguments, the LCA object defined before, the limit of the impacts by fraction of the most damaging biosphere flows and the type of limit we want. It returns a list of tuples: (lca score, inventory amount, flow name).

Let's apply this to our example:

In [84]:
results_bio_flows = ca.annotated_top_emissions(myFirstLCA, limit=0.05, limit_type='percent')
results_bio_flows

[(0.0018429979723834586,
  0.0018429979723834588,
  'Carbon dioxide, fossil' (kilogram, None, ('air',))),
 (0.0005411582533321144,
  0.0005411582533321144,
  'Carbon dioxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks'))),
 (0.00039838117671860726,
  0.0003983811767186072,
  'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground'))),
 (0.000231800725285436,
  8.2785973316227e-06,
  'Methane, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')))]

In [85]:
df = pd.DataFrame(results_bio_flows, columns=['CC score', 'Inventory amount', 'Flow Name'])
df

,CC score,Inventory amount,Flow Name
0,0.001843,0.001843,"[categories, code, CAS number, synonyms, name,..."
1,0.000541,0.000541,"[categories, code, CAS number, synonyms, name,..."
2,0.000398,0.000398,"[categories, code, CAS number, synonyms, name,..."
3,0.000232,0.000008,"[categories, code, CAS number, synonyms, name,..."


Look at the first three tuples of the list:

**QUESTION:** Why, for each flow, is the LCA score and the amount of flow equal?

## 3.2) Single-LCA: LCA with an activity from the foreground database related to a pavement system <a name="single_lca2"></a>

In the previous example we saw how to calculate the environmental impacts of an activity existing in the ecoinvent. In this example, we will see how to calculate the environmental impacts of a foreground activity defined in the foreground database 'Asphalt', introduced in the notebook '2b_Foreground', and stores in the excel file "Foreground.xlsx". Specifically, we will select the activity 'crushed stone, production and transport'. As depicted in the flowchat bellow, this activity has as inputs the exchanges 'gravel production, crushed', 'market for transport, freight, lorry, unspecified' and 'market for transport, freight, inland waterways, barge' (those are outputs of background activities), and as output the exchange 'crushed stone' (which is the reference product).


<img src="Flowchart_Crushed stone, production and transport.jpg" style ="width:750px;height:200px"/>

In [38]:
my_act = fgdb.get("crushed_stone") #the argument of the method is the code of the activity we are interested in
my_act

'crushed stone, production and transport' (kilogram, NL, None)

We can see the names of the activities whose reference products are inputs to our activity by seeing the activities that belong to the first level of the supply chain:

In [27]:
bwa.print_recursive_supply_chain(my_act, max_level=1)

1: 'crushed stone, production and transport' (kilogram, NL, None)
  1: 'gravel production, crushed' (kilogram, RoW, None)
  0.025: 'market for transport, freight, lorry, unspecified' (ton kilometer, RER, None)
  0.66: 'market for transport, freight, inland waterways, barge' (ton kilometer, RER, None)


### Functional unit

Let's assume we are interested in the production of 1 unit of the reference product (i.e., crushed stone)

In [28]:
functional_unit = {my_act: 1}
functional_unit

{'crushed stone, production and transport' (kilogram, NL, None): 1}

### LCIA Methods

The next step consists of selecting the LCIA method to be used for calculating the environmental impact scores. In the notebook '2a_Background' we explored the different LCIA methods that come with the databse 'biosphere3'. In this example, we are going to use the method 'CML v4.8 2016', and particularly the impact category 'Acification'.

In [29]:
CML_methods = [m for m in bd.methods if m[0] == "CML v4.8 2016"]
CML_methods

[('CML v4.8 2016',
  'acidification',
  'acidification (incl. fate, average Europe total, A&B)'),
 ('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)'),
 ('CML v4.8 2016',
  'ecotoxicity: freshwater',
  'freshwater aquatic ecotoxicity (FAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: marine',
  'marine aquatic ecotoxicity (MAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: terrestrial',
  'terrestrial ecotoxicity (TETP inf)'),
 ('CML v4.8 2016',
  'energy resources: non-renewable',
  'abiotic depletion potential (ADP): fossil fuels'),
 ('CML v4.8 2016', 'eutrophication', 'eutrophication (fate not incl.)'),
 ('CML v4.8 2016', 'human toxicity', 'human toxicity (HTP inf)'),
 ('CML v4.8 2016',
  'material resources: metals/minerals',
  'abiotic depletion potential (ADP): elements (ultimate reserves)'),
 ('CML v4.8 2016',
  'ozone depletion',
  'ozone layer depletion (ODP steady state)'),
 ('CML v4.8 2016',
  'photochemical oxidant formation',
  'photochemical oxidation (h

The impact category we are interested in is defined as following:

In [30]:
Acid_method = ('CML v4.8 2016', 'acidification', 'acidification (incl. fate, average Europe total, A&B)')

### LCIA scores (i.e. results) calculation

Next, we instantiate our LCA object:

In [31]:
mySecondLCA = bc.LCA(functional_unit, Acid_method)

And do the LCI and LCIA calculations:

In [32]:
mySecondLCA.lci()
mySecondLCA.lcia()

Finally, we can print the LCA score:

In [33]:
print("The score is {:f} {} for impact category {}".format(mySecondLCA.score, 
                                                 bd.methods.get(Acid_method).get('unit'),
                                                 Acid_method[1]))

The score is 0.000269 kg SO2-Eq for impact category acidification


In [34]:
df = mySecondLCA.to_dataframe()
df

,row_index,col_index,amount,row_id,col_id,row_database,row_code,row_name,row_location,row_unit,row_type,row_categories,row_product,col_database,col_code,col_name,col_location,col_unit,col_type,col_reference_product
0,2402,18253,1.551463e-04,4691,22963,biosphere3,77357947-ccc5-438e-9996-95e65e1e1bce,Nitrogen oxides,None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,e43dd9584a807e73dbbcd9d12f26a35e,"transport, freight, inland waterways, barge",RER,ton kilometer,process,"transport, freight, inland waterways, barge"
1,2405,14277,9.804713e-06,4694,18987,biosphere3,c1b91234-6f24-417b-8309-46111d09c457,Nitrogen oxides,None,kilogram,emission,air,None,ecoinvent-391-cutoff,0c8f3330261952e02d86355874fd2dd8,"diesel, burned in building machine",GLO,megajoule,process,"diesel, burned in building machine"
2,946,6871,8.251856e-06,1995,11581,biosphere3,fd7aa71c-508c-480d-81a6-8052aad92646,Sulfur dioxide,None,kilogram,emission,air,None,ecoinvent-391-cutoff,f4e8f9e773206c58d4a9ce05bdfb1e69,"diesel production, petroleum refinery operation",Europe without Switzerland,kilogram,process,diesel
3,944,18253,4.468214e-06,1992,22963,biosphere3,78c3efe4-421c-4d30-82e4-b97ac5124993,Sulfur dioxide,None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,e43dd9584a807e73dbbcd9d12f26a35e,"transport, freight, inland waterways, barge",RER,ton kilometer,process,"transport, freight, inland waterways, barge"
4,944,20007,3.612127e-06,1992,24717,biosphere3,78c3efe4-421c-4d30-82e4-b97ac5124993,Sulfur dioxide,None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,aebb8b0044d096580f717ac29b5dce3f,"transport, freight, sea, tanker for petroleum",GLO,ton kilometer,process,"transport, freight, sea, tanker for petroleum"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2402,6076,8.140939e-08,4691,10786,biosphere3,77357947-ccc5-438e-9996-95e65e1e1bce,Nitrogen oxides,None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,d24bedfbe8f0bc3405059dd13f83c3fb,"electricity production, hard coal",CN-GS,kilowatt hour,process,"electricity, high voltage"
196,2402,13102,8.095730e-08,4691,17812,biosphere3,77357947-ccc5-438e-9996-95e65e1e1bce,Nitrogen oxides,None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,63bc9f3b9cf61fb5f2b2c2861fde4fd8,"heat and power co-generation, lignite",RU,kilowatt hour,process,"electricity, high voltage"
197,944,10157,7.951846e-08,1992,14867,biosphere3,78c3efe4-421c-4d30-82e4-b97ac5124993,Sulfur dioxide,None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,d47c6f40c946e5915657ec3aa90b2c96,"electricity production, hard coal",CN-YN,kilowatt hour,process,"electricity, high voltage"
198,2402,14331,7.948846e-08,4691,19041,biosphere3,77357947-ccc5-438e-9996-95e65e1e1bce,Nitrogen oxides,None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,798097af2b9c00dc3e391f82d8c7b9d5,"transport, freight, sea, tanker for liquefied ...",GLO,ton kilometer,process,"transport, freight, sea, tanker for liquefied ..."


### Contribution analysis

In [35]:
ca = bwa.ContributionAnalysis()
results = ca.annotated_top_processes(mySecondLCA, limit=20) #returns a list of tuples: (lca score, supply amount, activity name)

print("Impact category ** {} **".format(Acid_method[1].title()))
print(results)

Impact category ** Acidification **
[(0.00016012896746236, 0.6601971394768142, 'transport, freight, inland waterways, barge' (ton kilometer, RER, None)), (1.035810226525831e-05, 0.01903827659848328, 'diesel, burned in building machine' (megajoule, GLO, None)), (9.00833652549729e-06, 0.006299162417008923, 'diesel production, petroleum refinery operation' (kilogram, Europe without Switzerland, None)), (5.911099075056097e-06, 0.04295536672686118, 'transport, freight, sea, tanker for petroleum' (ton kilometer, GLO, None)), (3.4595308283463968e-06, 0.002955528283951732, 'clinker production' (kilogram, RoW, None)), (2.598923349364906e-06, 0.017867251810533884, 'transport, freight, sea, bulk carrier for dry goods' (ton kilometer, GLO, None)), (2.451782366694148e-06, 0.002802036929741194, 'heat production, at hard coal industrial furnace 1-10MW' (megajoule, RoW, None)), (2.3342125167467973e-06, 4.8575991311652253e-05, 'electricity production, hard coal, at coal mine power plant' (kilowatt hour

In [36]:
df = pd.DataFrame(results, columns=['CC score', 'Supply amount', 'Activity Name'])
df

,CC score,Supply amount,Activity Name
0,0.000160,0.660197,"[comment, classifications, activity type, acti..."
1,0.000010,0.019038,"[comment, classifications, activity type, acti..."
2,0.000009,0.006299,"[comment, classifications, activity type, acti..."
3,0.000006,0.042955,"[comment, classifications, activity type, acti..."
4,0.000003,0.002956,"[comment, classifications, activity type, acti..."
5,0.000003,0.017867,"[comment, classifications, activity type, acti..."
6,0.000002,0.002802,"[comment, classifications, activity type, acti..."
7,0.000002,0.000049,"[comment, classifications, activity type, acti..."
8,0.000002,0.003583,"[comment, classifications, activity type, acti..."
9,0.000002,0.000011,"[comment, classifications, activity type, acti..."


## 3.3) Single-LCA: Revisiting my second LCA with multiple impact categories <a name="single_lca2"></a>

In the previous example we saw how to calculate the environmental impact scores for one impact category, namely 'Acidification'. In this example we will see how to calculate the environmental impact scores for multiple impact categories

### Product system

The initial step (i.e., Product system definition) is the same as in the previous example. The differences start in the selection of the LCIA methods

In [39]:
my_act = fgdb.get("crushed_stone") #the argument of the method is the code of the activity we are interested in
my_act

'crushed stone, production and transport' (kilogram, NL, None)

### Functional Unit

Let's assume we are interested in the production of 1 unit of the reference product (i.e., crushed stone)

In [40]:
functional_unit = {my_act:1}

### LCIA Method

Let's continue to use the method 'CML v4.8 2016'. To select all impact categories, we can use a piece of code that you are already familiar with:

In [41]:
cml_methods = [m for m in bd.methods if m[0] == "CML v4.8 2016"]
cml_methods

[('CML v4.8 2016',
  'acidification',
  'acidification (incl. fate, average Europe total, A&B)'),
 ('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)'),
 ('CML v4.8 2016',
  'ecotoxicity: freshwater',
  'freshwater aquatic ecotoxicity (FAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: marine',
  'marine aquatic ecotoxicity (MAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: terrestrial',
  'terrestrial ecotoxicity (TETP inf)'),
 ('CML v4.8 2016',
  'energy resources: non-renewable',
  'abiotic depletion potential (ADP): fossil fuels'),
 ('CML v4.8 2016', 'eutrophication', 'eutrophication (fate not incl.)'),
 ('CML v4.8 2016', 'human toxicity', 'human toxicity (HTP inf)'),
 ('CML v4.8 2016',
  'material resources: metals/minerals',
  'abiotic depletion potential (ADP): elements (ultimate reserves)'),
 ('CML v4.8 2016',
  'ozone depletion',
  'ozone layer depletion (ODP steady state)'),
 ('CML v4.8 2016',
  'photochemical oxidant formation',
  'photochemical oxidation (h

### LCIA scores (i.e. results) calculation¶

In this step we are going to calculate the environmental impact scores. To do so, we need to do the following:

In [42]:
# Create a lca object
myThirdLca = bc.LCA(functional_unit)

# Builds matrices, solves the system, generates an LCI matrix.
myThirdLca.lci()

results = [] #this list will store the LCA scores
for method in cml_methods:
    myThirdLca.switch_method(method) # the switch method is used to move to the next impact category in the list after the completation of one loop
    myThirdLca.lcia() # Characterization, i.e. the multiplication of the elements of the LCI matrix with characterization factors from the chosen LCIA method
    results.append((method[1].title(), 
                    myThirdLca.score, # Returns the score, i.e. the sum of the characterized inventory
                    bd.methods.get(method).get('unit')))
    print("The score is {:f} {} for the impact category {}".format(myThirdLca.score, 
                                                 bd.methods.get(method).get('unit'),
                                                 method[1].title())
         )

The score is 0.000269 kg SO2-Eq for the impact category Acidification
The score is 0.046649 kg CO2-Eq for the impact category Climate Change
The score is 0.013422 kg 1,4-DCB-Eq for the impact category Ecotoxicity: Freshwater
The score is 27.750110 kg 1,4-DCB-Eq for the impact category Ecotoxicity: Marine
The score is 0.000135 kg 1,4-DCB-Eq for the impact category Ecotoxicity: Terrestrial
The score is 0.552126 megajoule for the impact category Energy Resources: Non-Renewable
The score is 0.000076 kg PO4-Eq for the impact category Eutrophication
The score is 0.025743 kg 1,4-DCB-Eq for the impact category Human Toxicity
The score is 0.000000 kg Sb-Eq for the impact category Material Resources: Metals/Minerals
The score is 0.000000 kg CFC-11-Eq for the impact category Ozone Depletion
The score is 0.000019 kg ethylene-Eq for the impact category Photochemical Oxidant Formation


We can also see the results in a table

In [43]:
#Creates a table/Dataframw with the results
results_df = pd.DataFrame(results, columns=["Impact Category", "Score", "Unit"])
results_df = results_df.set_index("Impact Category")
results_df

,Score,Unit
Impact Category,,
Acidification,2.688610e-04,kg SO2-Eq
Climate Change,4.664932e-02,kg CO2-Eq
Ecotoxicity: Freshwater,1.342170e-02,"kg 1,4-DCB-Eq"
Ecotoxicity: Marine,2.775011e+01,"kg 1,4-DCB-Eq"
Ecotoxicity: Terrestrial,1.346317e-04,"kg 1,4-DCB-Eq"
Energy Resources: Non-Renewable,5.521265e-01,megajoule
Eutrophication,7.598996e-05,kg PO4-Eq
Human Toxicity,2.574321e-02,"kg 1,4-DCB-Eq"
Material Resources: Metals/Minerals,1.020628e-07,kg Sb-Eq


### Contribution Analysis

To perform the CA we run the cell below:

In [44]:
#Initialize the dictionary that will store the results
results_dic= {}

for method in cml_methods:
    myThirdLca.switch_method(method) #the switch method is used to move to the next impact category in the list after the completation of one loop
    myThirdLca.lcia() # Characterization, i.e. the multiplication of the elements of the LCI matrix with characterization factors from the chosen LCIA method
    
    ca = bwa.ContributionAnalysis()
    results = ca.annotated_top_processes(myThirdLca, limit=5) #returns a list of tuples: (lca score, supply amount, activity name)

    print("Impact category '{}'".format(method[1].title()))
    print(results)    
    print("")

    results_dic[method[1].title()] = results


Impact category 'Acidification'
[(0.00016012896746236, 0.6601971394768142, 'transport, freight, inland waterways, barge' (ton kilometer, RER, None)), (1.035810226525831e-05, 0.01903827659848328, 'diesel, burned in building machine' (megajoule, GLO, None)), (9.00833652549729e-06, 0.006299162417008923, 'diesel production, petroleum refinery operation' (kilogram, Europe without Switzerland, None)), (5.911099075056097e-06, 0.04295536672686118, 'transport, freight, sea, tanker for petroleum' (ton kilometer, GLO, None)), (3.4595308283463968e-06, 0.002955528283951732, 'clinker production' (kilogram, RoW, None))]

Impact category 'Climate Change'
[(0.02009009606272066, 0.6601971394768142, 'transport, freight, inland waterways, barge' (ton kilometer, RER, None)), (0.002730329779653154, 0.00016654446014661243, 'natural gas venting from petroleum/natural gas production' (cubic meter, GLO, None)), (0.0023905016627848562, 0.002955528283951732, 'clinker production' (kilogram, RoW, None)), (0.0014059

Let's now visualize the results with DataFrames

In [48]:
# Initialize an empty list to store DataFrames
dfs = {}

# Loop over the dictionary
for impact_category, impact_score in results_dic.items():
    # Convert the list of tuples into a DataFrame
    df = pd.DataFrame(impact_score, columns=['Score', 'Supply Amount', 'Activity'])
    
    # Store the DataFrame in the dictionary with the impact category as the key
    dfs[impact_category] = df

# Efficiently display all DataFrames with their category names
for category, df in dfs.items():
    print(f"{category}:")
    print(df)
    print("\n" + "-"*40 + "\n")  # Separator between DataFrames

Acidification:
      Score  Supply Amount                                           Activity
0  0.000160       0.660197  [comment, classifications, activity type, acti...
1  0.000010       0.019038  [comment, classifications, activity type, acti...
2  0.000009       0.006299  [comment, classifications, activity type, acti...
3  0.000006       0.042955  [comment, classifications, activity type, acti...
4  0.000003       0.002956  [comment, classifications, activity type, acti...

----------------------------------------

Climate Change:
      Score  Supply Amount                                           Activity
0  0.020090       0.660197  [comment, classifications, activity type, acti...
1  0.002730       0.000167  [comment, classifications, activity type, acti...
2  0.002391       0.002956  [comment, classifications, activity type, acti...
3  0.001406       0.019038  [comment, classifications, activity type, acti...
4  0.001233       0.006299  [comment, classifications, activity type

## 3.4) Single-LCA: LCA of a pavement system encompassing the pavement life cycle phases A1-A5 <a name="single_lca4"></a>

So far we have seen how to calculate the environmental impacts of single background and foreground activities. In this example, we will see how to calculate the environmental impacts related to the mixtures production, materials transportation and construction of the Initial pavement structure defined in the file "Case study lectures". This is an example similar to what you will have to do in your group assignment.

### Product system

Our product system comprises all activities needed to construct the initial pavement structure. There are multiple ways to define the activities needed to build the product system. In the excel file with the foreground database you can find all activities needed to "build" our product system (and others that not needed for this specific example but that will be useful for your group assignment). If you analyse their content, you will see that they are defined in such a way that we are moving downstream in the supply chain of the pavement system. In other words, the exchange output of an activity is an exchange input of other activity located dowstream that activity, all the way up to the activity 'pavement structure, AC Surf, 30% RAP, regular bitumen' that encampsulate the whole product system we are interested in. 


**EXERCISE:** Draw the flowchart with the activities belonging to the system boundaries of the product system.

Apart from the reference product of the activity 'pavement structure, AC Surf, 30% RAP, regular bitumen', each exchange input corresponds to a pavement layer. With this regard, you will see that there are two exchange inputs with the same name (AC Bin). This happen because the same type of mixture is used in the binder and base layers, despite the fact they have different names in the file "Case study lectures". It is also worth mentioning that for all activities the amount of input exchanges is the one needed to produce 1 unit of the respective reference product. Thus, the amount of the exchange inputs of the activity representing our product system is obtained by multiplying the density of the respective asphalt mixture by the dimensions of the layer where it is used. For instance, the mixture used in the surface layer has a density of 2350 kg/m3 (see column "comments in the foreground database). Given that the width of our pavement structure is 6 lanes * 3.75m/lane, the length is 1km and the thickness of the layer is 0.038m, then the total amount of the reference product is 6 * 3.75 * 10000 * 0.038 * 2350 = 2009250 kg.

In [2]:
my_act = fgdb.get("pavement_structure_1") #the argument of the method is the code of the activity we are interested in
my_act

NameError: name 'fgdb' is not defined

### Functional Unit

Given that the amount of exchange inputs are already defined for the dimensions of the pavement structure, the quantity of our reference product is 1. If the pavement section was 2 km long, or had 12 lanes, then we should define 2 units of the reference product, and so on...

In [55]:
functional_unit = {my_act:1}

### LCIA Method

Let's continue to use the method 'CML v4.8 2016'. 

In [56]:
cml_methods = [m for m in bd.methods if m[0] == "CML v4.8 2016"]
cml_methods

[('CML v4.8 2016',
  'acidification',
  'acidification (incl. fate, average Europe total, A&B)'),
 ('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)'),
 ('CML v4.8 2016',
  'ecotoxicity: freshwater',
  'freshwater aquatic ecotoxicity (FAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: marine',
  'marine aquatic ecotoxicity (MAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: terrestrial',
  'terrestrial ecotoxicity (TETP inf)'),
 ('CML v4.8 2016',
  'energy resources: non-renewable',
  'abiotic depletion potential (ADP): fossil fuels'),
 ('CML v4.8 2016', 'eutrophication', 'eutrophication (fate not incl.)'),
 ('CML v4.8 2016', 'human toxicity', 'human toxicity (HTP inf)'),
 ('CML v4.8 2016',
  'material resources: metals/minerals',
  'abiotic depletion potential (ADP): elements (ultimate reserves)'),
 ('CML v4.8 2016',
  'ozone depletion',
  'ozone layer depletion (ODP steady state)'),
 ('CML v4.8 2016',
  'photochemical oxidant formation',
  'photochemical oxidation (h

### LCIA scores (i.e. results) calculation¶

In this step we are going to calculate the environmental impact scores. To do so, we need to do the following:

In [57]:
# Create a lca object
myFourthLca = bc.LCA(functional_unit)

# Builds matrices, solves the system, generates an LCI matrix.
myFourthLca.lci()

results = [] #this list will store the LCA scores
for method in cml_methods:
    myFourthLca.switch_method(method) # the switch method is used to move to the next impact category in the list after the completation of one loop
    myFourthLca.lcia() # Characterization, i.e. the multiplication of the elements of the LCI matrix with characterization factors from the chosen LCIA method
    results.append((method[1].title(), 
                    myFourthLca.score, # Returns the score, i.e. the sum of the characterized inventory
                    bd.methods.get(method).get('unit')))
    print("The score is {:f} {} for the impact category {}".format(myFourthLca.score, 
                                                 bd.methods.get(method).get('unit'),
                                                 method[1].title())
         )

The score is 3935.726254 kg SO2-Eq for the impact category Acidification
The score is 1295038.759926 kg CO2-Eq for the impact category Climate Change
The score is 280050.811156 kg 1,4-DCB-Eq for the impact category Ecotoxicity: Freshwater
The score is 493693193.309577 kg 1,4-DCB-Eq for the impact category Ecotoxicity: Marine
The score is 4208.696764 kg 1,4-DCB-Eq for the impact category Ecotoxicity: Terrestrial
The score is 33404117.008079 megajoule for the impact category Energy Resources: Non-Renewable
The score is 963.613210 kg PO4-Eq for the impact category Eutrophication
The score is 101509736.400397 kg 1,4-DCB-Eq for the impact category Human Toxicity
The score is 2.073477 kg Sb-Eq for the impact category Material Resources: Metals/Minerals
The score is 0.050775 kg CFC-11-Eq for the impact category Ozone Depletion
The score is 372.108108 kg ethylene-Eq for the impact category Photochemical Oxidant Formation


In [58]:
#Creates a table with the results
results_df = pd.DataFrame(results, columns=["Impact Category", "Score", "Unit"])
results_df = results_df.set_index("Impact Category")
results_df

,Score,Unit
Impact Category,,
Acidification,3.935726e+03,kg SO2-Eq
Climate Change,1.295039e+06,kg CO2-Eq
Ecotoxicity: Freshwater,2.800508e+05,"kg 1,4-DCB-Eq"
Ecotoxicity: Marine,4.936932e+08,"kg 1,4-DCB-Eq"
Ecotoxicity: Terrestrial,4.208697e+03,"kg 1,4-DCB-Eq"
Energy Resources: Non-Renewable,3.340412e+07,megajoule
Eutrophication,9.636132e+02,kg PO4-Eq
Human Toxicity,1.015097e+08,"kg 1,4-DCB-Eq"
Material Resources: Metals/Minerals,2.073477e+00,kg Sb-Eq


### Contribution Analysis

Let's perfom a CA:

In [59]:
#Initialize the dictionary that will store the results
results_dic= {}

for method in cml_methods:
    myFourthLca.switch_method(method) #the switch method is used to move to the next impact category in the list after the completation of one loop
    myFourthLca.lcia() # Characterization, i.e. the multiplication of the elements of the LCI matrix with characterization factors from the chosen LCIA method
    
    ca = bwa.ContributionAnalysis()
    results = ca.annotated_top_processes(myFourthLca, limit=5) #returns a list of tuples: (lca score, supply amount, activity name)

    print("Impact category '{}'".format(method[1].title()))
    print(results)   
    print("")

    results_dic[method[1].title()] = results


Impact category 'Acidification'
[(1021.1799817224556, 4210231.999295342, 'transport, freight, inland waterways, barge' (ton kilometer, RER, None)), (549.7665008669605, 420854.03310056473, 'pitch production, petroleum refinery operation' (kilogram, Europe without Switzerland, None)), (345.52465098369163, 2510893.136406166, 'transport, freight, sea, tanker for petroleum' (ton kilometer, GLO, None)), (330.08727366992997, 606703.1061129088, 'diesel, burned in building machine' (megajoule, GLO, None)), (76.6483330955252, 526947.0793011605, 'transport, freight, sea, bulk carrier for dry goods' (ton kilometer, GLO, None))]

Impact category 'Climate Change'
[(332304.4530722553, 5629014.842366323, 'heat production, natural gas, at industrial furnace >100kW' (megajoule, Europe without Switzerland, None)), (181525.52614811345, 11072.681029398527, 'natural gas venting from petroleum/natural gas production' (cubic meter, GLO, None)), (128119.25446876982, 4210231.999295342, 'transport, freight, inla

Let's now visualize the results with DataFrames:

In [61]:
# Initialize an empty list to store DataFrames
dfs = {}

# Loop over the dictionary
for impact_category, impact_score in results_dic.items():
    # Convert the list of tuples into a DataFrame
    df = pd.DataFrame(impact_score, columns=['Score', 'Supply Amount', 'Activity'])
    
    # Store the DataFrame in the dictionary with the impact category as the key
    dfs[impact_category] = df

# Efficiently display all DataFrames with their category names
for category, df in dfs.items():
    print(f"{category}:")
    print(df)
    print("\n" + "-"*40 + "\n")  # Separator between DataFrames


Acidification:
         Score  Supply Amount  \
0  1021.179982   4.210232e+06   
1   549.766501   4.208540e+05   
2   345.524651   2.510893e+06   
3   330.087274   6.067031e+05   
4    76.648333   5.269471e+05   

                                            Activity  
0  [comment, classifications, activity type, acti...  
1  [comment, classifications, activity type, acti...  
2  [comment, classifications, activity type, acti...  
3  [comment, classifications, activity type, acti...  
4  [comment, classifications, activity type, acti...  

----------------------------------------

Climate Change:
           Score  Supply Amount  \
0  332304.453072   5.629015e+06   
1  181525.526148   1.107268e+04   
2  128119.254469   4.210232e+06   
3   72212.453577   4.208540e+05   
4   59532.581272   8.889938e+05   

                                            Activity  
0  [comment, classifications, activity type, acti...  
1  [comment, classifications, activity type, acti...  
2  [comment, classifi

## 3.5) Comparative-LCA of 2 pavement systems with MultiLCA <a name="comparative_lca"></a>

In this last example we are going to compare the environmental performance of the previous system with that of a second system that differs from the previous one in the type of mixtures used in the surface layer. Specifically, the mixture to be employed in the surface layer will use a modified bitumen.

To do so, we will use the `MultiLCA` functionality. It allows to calculate the LCA scores for multiple functional units and impact categories. To do so, we need to create a `calculation setup`, i.e. a named set of functional units and LCIA methods.


### Product system

The second product system is represented by the foreground activity 'pavement structure, AC Surf, 30% RAP, modified bitumen'. The first product system continues to be represented by the activity 'pavement structure, AC Surf, 30% RAP, regular bitumen'

In [62]:
#Pavement system 1
my_act1 = fgdb.get("pavement_structure_1") 

#Pavement system 2
my_act2 = fgdb.get("pavement_structure_2") 

### Functional Units

Given that the amount of exchange inputs are already defined for the dimensions of the pavement structure, the quantity of our reference products is 1.

In [63]:
#Pavement system 1
functional_unit1 = {my_act1:1}

#Pavement system 2
functional_unit2 = {my_act2:1}

# List of functional units
list_functional_units = [{my_act:1}, {my_act2:1}] #NOTE: in this example we are only considering two FU, but you can add as much as you want (need)

### LCIA Method

Let's continue to use the method 'CML v4.8 2016'. 

In [64]:
cml_methods = [m for m in bd.methods if m[0] == "CML v4.8 2016"]

# List of methods
list_methods = cml_methods

### LCIA scores (i.e. results) calculation¶

In this step we are going to calculate the environmental impact scores. To do so, we need to create a `calculation setup`, which is a dictionary with lists of functional units and methods. And we will name it 'Pavement system 1 vs Pavement system 2'

In [65]:
bd.calculation_setups['Pavement system 1 vs Pavement system 2'] = {'inv':list_functional_units, 'ia':list_methods}

Next, we create a `MultiLCA` object:

In [66]:
myMultiLCA = bc.MultiLCA('Pavement system 1 vs Pavement system 2')

Finally, we calculate the environmental impact scores and present them in a dataframe:

In [67]:
scores = pd.DataFrame(myMultiLCA.results, columns=myMultiLCA.methods)

as_activities = [
    (bc.get_activity(key), amount) 
    for dct in myMultiLCA.func_units 
    for key, amount in dct.items()
]
nicer_fu = pd.DataFrame(
    [
        (x['database'], x['code'], x['name'], x['location'], x['unit'], y) 
        for x, y in as_activities
    ], 
    columns=('Database', 'Code', 'Name', 'Location', 'Unit', 'Amount')
)
pd.concat([nicer_fu, scores], axis=1).T

,0,1
Database,asphalt,asphalt
Code,pavement_structure_1,pavement_structure_2
Name,"pavement structure, AC Surf, 30% RAP, regular ...","pavement structure, AC Surf, 30% RAP, modified..."
Location,NL,NL
Unit,kilometer,kilometer
Amount,1,1
"(CML v4.8 2016, acidification, acidification (incl. fate, average Europe total, A&B))",3935.726254,3980.361764
"(CML v4.8 2016, climate change, global warming potential (GWP100))",1295038.759926,1313454.135258
"(CML v4.8 2016, ecotoxicity: freshwater, freshwater aquatic ecotoxicity (FAETP inf))",280050.811156,287400.6826
"(CML v4.8 2016, ecotoxicity: marine, marine aquatic ecotoxicity (MAETP inf))",493693193.309577,507064252.405141
